In [1]:
!pip install kaggle

  Using cached kaggle-1.6.12-py3-none-any.whl
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached bleach-6.1.0-py3-none-any.whl.metadata (30 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
Using cached bleach-6.1.0-py3-none-any.whl (162 kB)
Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached urllib3-2.2.1-py3-none-an

In [2]:
!kaggle datasets download -d andrewmvd/sp-500-stocks -p data/market_data

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks
License(s): CC0-1.0
100%|███████████████████████████████████████| 42.7M/42.7M [01:41<00:00, 298kB/s]
100%|███████████████████████████████████████| 42.7M/42.7M [01:41<00:00, 442kB/s]


Then unzip and rename folder to `sp500`.

In [4]:
import numpy as np
import pandas as pd

In [5]:
# models
from models import FamaFrench3Factor, FamaFrench5Factor, Carhart